In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from statistics import mean

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from custom_classes import NasaDataset, SimpleAE, split_dataset, fix_seeds
from ae_metrics import MAPE
from rul_metrics import RMSELoss
from normalisation import standard_scaling, min_max_scaling

import os
import time

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'{device=}')

device='cuda'


### Training

In [3]:
train_dataset = NasaDataset('../datasets/clean_train_data.csv')
fix_seeds(37)
train_dataset, test_dataset = split_dataset(train_dataset, test_size=0.1)
fix_seeds(37)
train_dataset, val_dataset = split_dataset(train_dataset, test_size=0.3)
# test_dataset = NasaDataset('../datasets/test.csv')

norm_func = min_max_scaling
try:
    norm_name = repr(norm_func).split(' ', maxsplit=2)[1]
except IndexError:
    norm_name = 'no_scaling'
for dtset in (train_dataset, val_dataset, test_dataset):
    dtset.to(device)
    if norm_func:
        norm_func(dtset)

fix_seeds(37)
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
fix_seeds(37)
test_loader = DataLoader(val_dataset, batch_size=20, shuffle=True)
fix_seeds(37)
val_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=..., shuffle=...)
print(f'Train: {len(train_dataset)}\nValidation: {len(val_dataset)}\nTest: {len(test_dataset)}')

input_shape = train_dataset.get_input_shape()
layers_sizes = (8, 4, 2)

model_ae = SimpleAE(input_shape, layers_sizes).to(device)
loss_func = nn.MSELoss()
metric_func = MAPE()
optimiser = optim.Adam(model_ae.parameters(),
                       lr=1e-3)

Train: 13723
Validation: 4840
Test: 2068


In [4]:
def plot_history(history_df: pd.DataFrame, ylabel: str = None, save_path: str = None):
    history_df = history_df.melt(ignore_index=False).iloc[1:]
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 5)
    sns.lineplot(data=history_df,
                x=history_df.index,
                y='value',
                hue='variable')

    ax.set_ylabel(ylabel)

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path)
    plt.show()

In [5]:
def check_path(pth):
    if not os.path.exists(pth):
        os.makedirs(pth)
    return pth

In [6]:
epochs = 100
history = list()

for epoch in range(epochs):
    train_losses = list()
    train_metrics = list()
    for dta in train_loader:
        sample = dta['sensors']
        sample = sample.to(device)
        _, reconstruction = model_ae(sample)

        loss = loss_func(reconstruction, sample)
        metric = metric_func(reconstruction, sample)

        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        train_losses.append(loss.item())
        train_metrics.append(metric.item())
    
    with torch.no_grad():
        val_losses = list()
        val_metrics = list()
        for dta in val_loader:
            sample = dta['sensors']
            sample = sample.to(device)
            _, reconstruction = model_ae(sample)

            loss = loss_func(reconstruction, sample)
            metric = metric_func(reconstruction, sample)

            val_losses.append(loss.item())
            val_metrics.append(metric.item())
    
    train_loss, val_loss = mean(train_losses), mean(val_losses)
    train_metrics, val_metrics = mean(train_metrics), mean(val_metrics)
    history.append((epoch, train_loss, val_loss, train_metrics, val_metrics))
    if (epoch + 1) % 10 == 0 or epoch == epochs - 1:
        print(f'{epoch+1:>3}/{epochs:>3}: {train_loss=:.4f}, {val_loss=:.4f}, {train_metrics=:.4f}%, {val_metrics=:.4f}%')

with torch.no_grad():
    test_losses = list()
    test_metrics = list()
    for sample in test_loader:
        sample = dta['sensors']
        sample = sample.to(device)
        _, reconstruction = model_ae(sample)
        loss = loss_func(reconstruction, sample)
        metric = metric_func(reconstruction, sample)
        test_losses.append(loss.item())
        test_metrics.append(metric.item())
    test_loss = mean(test_losses)
    test_metrics = mean(test_metrics)
    print(f'\n{test_loss=:.4f}, {test_metrics=:.4f}%')


# -------------------------------------------------- #
name = str(layers_sizes)
models_path = '../Models/'
plot_path = '../Plots/history/'


if True:
    pth = check_path(os.path.join(models_path, norm_name))
    torch.save(model_ae.state_dict(), os.path.join(pth, f'{name}.pth'))

if True:
    columns = ('epoch', 'train_loss', 'val_loss', 'train_metric', 'val_metric')
    total_history_df = pd.DataFrame(history, columns=columns).set_index('epoch')

    history_df = total_history_df.loc[:,('train_loss', 'val_loss')]
    pth = check_path(os.path.join(plot_path, 'loss', norm_name))
    plot_history(history_df, ylabel='RMSE', save_path=os.path.join(pth, f'{name}.png'))

    pth = check_path(os.path.join(plot_path, 'metric', norm_name))
    history_df = total_history_df.loc[:,('train_metric', 'val_metric')]
    plot_history(history_df, ylabel='MAPE', save_path=os.path.join(pth, f'{name}.png'))

 10/100: train_loss=0.0067, val_loss=0.0066, train_metrics=inf%, val_metrics=inf%
 20/100: train_loss=0.0061, val_loss=0.0061, train_metrics=inf%, val_metrics=inf%
 30/100: train_loss=0.0059, val_loss=0.0059, train_metrics=inf%, val_metrics=inf%
 40/100: train_loss=0.0059, val_loss=0.0059, train_metrics=inf%, val_metrics=inf%
 50/100: train_loss=0.0059, val_loss=0.0059, train_metrics=inf%, val_metrics=inf%
